In [ ]:
pip install torch torchvision efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=5fbbac8acce5abd243a3d677bcc76c1919f727fdac9864686799499a6069764e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Define the Attention Block
class AttentionBlock(layers.Layer):
    def __init__(self, in_channels=1792):
        super(AttentionBlock, self).__init__()
        self.conv = layers.Conv2D(1, kernel_size=1)
        self.sigmoid = layers.Activation('sigmoid')

    def call(self, inputs):
        attention = self.sigmoid(self.conv(inputs))
        return inputs * attention

# Define the EfficientNetWithAttention Model
class EfficientNetWithAttention(models.Model):
    def __init__(self):
        super(EfficientNetWithAttention, self).__init__()
        self.base_model = EfficientNetB4(weights='imagenet', include_top=False)
        self.attention_block = AttentionBlock(in_channels=1792)
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.attention_block(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

# Define image preprocessing for TensorFlow/Keras
def load_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((224, 224))  # Resize to match EfficientNet input
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict
def predict_image(input_tensor, model, threshold=0.5):
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
    output = model(input_tensor)
    prob = tf.sigmoid(output).numpy().item()
    prediction = "Fake" if prob > threshold else "Real"
    return prediction, prob

# Example usage
if __name__ == "__main__":
    # Initialize model
    model = EfficientNetWithAttention()

    # Load and preprocess image
    image_path = "/content/Screenshot 2024-11-09 at 4.43.42 AM.png"  # Update with actual path
    input_tensor = load_image(image_path)

    # Make a prediction
    prediction, prob = predict_image(input_tensor, model)
    print(f"The image is classified as: {prediction}")
    print(f"Confidence score for 'fake': {prob:.4f}")




The image is classified as: Fake
Confidence score for 'fake': 0.6212
